In [1]:
# Prepare gpu for training
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

tf.config.list_physical_devices("GPU")


from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

2022-11-16 21:20:58.872085: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-16 21:20:58.951283: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-16 21:20:59.385417: W tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libnvinfer.so.8'; dlerror: libnvinfer.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/chen/miniconda3/lib/:/home/chen/miniconda3/envs/tf/lib/
2022-11-16 21:20:59.385673: W tensorflow/tsl/plat

1 Physical GPUs, 1 Logical GPU
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3070, compute capability 8.6


2022-11-16 21:20:59.859829: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2022-11-16 21:20:59.864864: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2022-11-16 21:20:59.865003: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
from scripts.model.build import get_architecture
from scripts.experiment import load_config

config = load_config("dcgan_scapes_16m", "model")

gen_summary, disc_summary = get_architecture(config)

print(gen_summary)

print(disc_summary)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 25600)             2585600   
                                                                 
 reshape_2 (Reshape)         (None, 5, 5, 1024)        0         
                                                                 
 re_lu_10 (ReLU)             (None, 5, 5, 1024)        0         
                                                                 
 batch_normalization_18 (Bat  (None, 5, 5, 1024)       4096      
 chNormalization)                                                
                                                                 
 up_sampling2d_16 (UpSamplin  (None, 10, 10, 1024)     0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 10, 10, 1024)     

## Train

In [2]:
from scripts.experiment import run_experiment
generator, discriminator = run_experiment("mini_scapes_resized", "dcgan_mini_scapes", "dcgan_mini_scapes_v1", num_examples_to_generate=4, epochs=1, colab=False)

Loss for previous batch #0: Generator loss = None, Discriminator loss = None
Epoch # 1/1
Batch # 1


2022-11-15 10:29:50.298961: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:421] Loaded runtime CuDNN library: 8.1.0 but source was compiled with: 8.6.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2022-11-15 10:29:50.299387: W ./tensorflow/compiler/xla/stream_executor/stream.h:1553] attempting to perform DNN operation using StreamExecutor without DNN support


InternalError: Graph execution error:

Detected at node 'sequential/batch_normalization/FusedBatchNormV3' defined at (most recent call last):
    File "/home/chen/miniconda3/envs/tf/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/chen/miniconda3/envs/tf/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/chen/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/chen/miniconda3/envs/tf/lib/python3.9/site-packages/traitlets/config/application.py", line 982, in launch_instance
      app.start()
    File "/home/chen/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/chen/miniconda3/envs/tf/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    Fi

## Save experiment results to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp -r /content/results/dcgan_mini_scapes_v1 /content/gdrive/experiments/dcgan_mini_scapes_v1

drive.flush_and_unmount()

In [1]:
# load model with tf keras

from tensorflow.keras.models import load_model

generator = load_model("/home/chen/big_bertha/coding/capstone/art-generator/experiments/models/generator_epoch_10")

/home/chen/miniconda3/envs/tf/lib/python3.9/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


/home/chen/miniconda3/envs/tf/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


InternalError: Exception encountered when calling layer 'batch_normalization_20' (type BatchNormalization).

{{function_node __wrapped__FusedBatchNormV3_device_/job:localhost/replica:0/task:0/device:GPU:0}} cuDNN launch failure : input shape ([1,5,5,512]) [Op:FusedBatchNormV3]

Call arguments received by layer 'batch_normalization_20' (type BatchNormalization):
  • inputs=tf.Tensor(shape=(1, 5, 5, 512), dtype=float16)
  • training=False
  • mask=None